# Debug continuous image decoding
I noticed that the results of continuous image decoding looked odd, even though the ROI decoding looked good.

Patterns I noticed from the results below:
- pFgA, pFgA_given_pF=0.50, pAgF, and pAgF_given_pF=0.50 all return the most common terms (e.g., task, using, magnetic).
- consistency_z also gives common terms, although a different set (e.g., motor, cerebellum, temporal).
    - The bottom terms seem relevant for faces, emotion, and reward, but not visual.
- specificity_z is a mixed bag:
    - faces has relevant terms at its *lowest* weights
    - emotion has some relevant terms (amygdala is highest; cingulate, negative) at highest weights
    - reward has relevant terms at highest weights
    - so does visual.

In [1]:
from os import mkdir
from os.path import join, isdir
from shutil import rmtree

import gclda
import neurosynth
import pandas as pd

In [2]:
# Use the current Neurosynth database (n=11406 studies, ~3k features)
d = '/Users/tsalo/Desktop/ns-dataset/'
d2 = '/Users/tsalo/Documents/tsalo/python_gclda/examples/gclda_results/20170728/'
dset = neurosynth.Dataset.load(join(d, 'updated_dataset.pkl'))

# Load GC-LDA model trained on current Neurosynth database
# (1000 iterations, 400 topics, 2 symmetric subregions per topic) 
model = gclda.Model.load(join(d2, 'model.pkl'))
decoder = gclda.Decoder(model)

names = ['faces', 'emotion', 'reward', 'vision']
terms = ['face* | facia*', 'emo*', 'reward*', 'visua* | visio*']

In [3]:
# Run meta-analyses
if isdir('temp'):
    rmtree('temp/')
mkdir('temp')
for i, term in enumerate(terms):
    name = names[i]
    ids = dset.get_studies(expression=term)
    print('{0}: {1} studies'.format(name, len(ids)))
    ma = neurosynth.meta.MetaAnalysis(dset, ids)
    ma.save_results('temp', name)

Generating LALR tables


faces: 1180 studies


/Users/tsalo/Documents/tsalo/neurosynth/neurosynth/analysis/meta.py:134: RuntimeWarning: invalid value encountered in divide
  pFgA = pAgF * pF / pA
/Users/tsalo/Documents/tsalo/neurosynth/neurosynth/analysis/meta.py:139: RuntimeWarning: invalid value encountered in divide
  pFgA_prior = pAgF * prior / pAgF_prior
Generating LALR tables


emotion: 1690 studies


Generating LALR tables


reward: 719 studies


Generating LALR tables


vision: 2719 studies


## Decode reverse inference z-values

In [4]:
dfs = []
for i, name in enumerate(names):
    meta_file = 'temp/{0}_specificity_z.nii.gz'.format(name)
    results = decoder.decode_continuous(meta_file)
    results = results.sort_values(by=['Weight'], ascending=False)
    results[name] = results.index
    results = results[[name, 'Weight']].reset_index(drop=True)
    dfs.append(results)
df = pd.concat(dfs, axis=1)
print('Top terms:')
print df.head(10)

print('\nBottom terms:')
df = df.reindex(index=df.index[::-1])
print df.head(10)

Top terms:
         faces    Weight        emotion    Weight         reward    Weight  \
0        motor  0.058412       amygdala  0.048428         reward  0.076418   
1     parietal  0.058229       temporal  0.031527     prefrontal  0.060080   
2         task  0.029451         medial  0.029453       striatum  0.058641   
3         hand  0.026893     prefrontal  0.029044      cingulate  0.052994   
4  performance  0.025487       anterior  0.026216       amygdala  0.052688   
5      network  0.024093      cingulate  0.018783  orbitofrontal  0.046582   
6    cognitive  0.022602       negative  0.016749      responses  0.045162   
7      control  0.020652      responses  0.016541       anterior  0.044982   
8    movements  0.018363  orbitofrontal  0.015805         medial  0.041469   
9    cingulate  0.018286         social  0.015560        ventral  0.041440   

        vision    Weight  
0       visual  0.075171  
1         task  0.038792  
2     parietal  0.027326  
3    occipital  0.0235

## Decode forward inference z-values

In [5]:
dfs = []
for i, name in enumerate(names):
    meta_file = 'temp/{0}_consistency_z.nii.gz'.format(name)
    results = decoder.decode_continuous(meta_file)
    results = results.sort_values(by=['Weight'], ascending=False)
    results[name] = results.index
    results = results[[name, 'Weight']].reset_index(drop=True)
    dfs.append(results)
df = pd.concat(dfs, axis=1)
print('Top terms:')
print df.head(10)

print('\nBottom terms:')
df = df.reindex(index=df.index[::-1])
print df.head(10)

Top terms:
          faces    Weight     emotion    Weight       reward    Weight  \
0         motor  6.642553       motor  1.613512     temporal  2.746502   
1    cerebellum  6.366671  cerebellum  1.586536   cerebellum  2.725885   
2    cerebellar  5.821531  cerebellar  1.428512        motor  2.676280   
3  connectivity  2.565251      memory  0.557191   cerebellar  2.523600   
4          hand  2.544607        hand  0.552967       memory  1.294360   
5     cognitive  2.475645     working  0.475088       visual  1.064248   
6    prefrontal  2.435233      verbal  0.448191         hand  0.923396   
7       healthy  2.146209   brainstem  0.442665      working  0.905417   
8     brainstem  2.056085   movements  0.425790  information  0.851446   
9      movement  1.999678    movement  0.421088    cognitive  0.844035   

       vision    Weight  
0        task  8.333492  
1      visual  8.000651  
2    parietal  4.661911  
3     stimuli  3.073305  
4       gyrus  2.904892  
5    response  2.3

## Decode reverse inference conditional probabilities

In [6]:
dfs = []
for i, name in enumerate(names):
    meta_file = 'temp/{0}_pFgA.nii.gz'.format(name)
    results = decoder.decode_continuous(meta_file)
    results = results.sort_values(by=['Weight'], ascending=False)
    results[name] = results.index
    results = results[[name, 'Weight']].reset_index(drop=True)
    dfs.append(results)
df = pd.concat(dfs, axis=1)
print('Top terms:')
print df.head(10)

print('\nBottom terms:')
df = df.reindex(index=df.index[::-1])
print df.head(10)

Top terms:
        faces    Weight     emotion    Weight      reward    Weight  \
0        task  0.008815        task  0.008119        task  0.008391   
1    temporal  0.008667    temporal  0.007869  prefrontal  0.007659   
2       using  0.006775  prefrontal  0.007206       using  0.006571   
3  prefrontal  0.006210       using  0.006800    magnetic  0.006249   
4      visual  0.006191    magnetic  0.006299   resonance  0.006150   
5    magnetic  0.006012   resonance  0.006072    temporal  0.005471   
6   resonance  0.005873    anterior  0.005430    anterior  0.005292   
7      memory  0.005536     frontal  0.005251     control  0.005121   
8     frontal  0.004952     network  0.005102     frontal  0.005015   
9     network  0.004870      memory  0.005091    response  0.004987   

       vision    Weight  
0        task  0.009956  
1      visual  0.007265  
2    temporal  0.006840  
3       using  0.006560  
4    magnetic  0.006147  
5   resonance  0.005984  
6    parietal  0.005689  

## Decode forward inference conditional probabilities

In [7]:
dfs = []
for i, name in enumerate(names):
    meta_file = 'temp/{0}_pAgF.nii.gz'.format(name)
    results = decoder.decode_continuous(meta_file)
    results = results.sort_values(by=['Weight'], ascending=False)
    results[name] = results.index
    results = results[[name, 'Weight']].reset_index(drop=True)
    dfs.append(results)
df = pd.concat(dfs, axis=1)
print('Top terms:')
print df.head(10)

print('\nBottom terms:')
df = df.reindex(index=df.index[::-1])
print df.head(10)

Top terms:
        faces    Weight     emotion    Weight      reward    Weight  \
0        task  0.010526        task  0.009765        task  0.010037   
1    temporal  0.007372    temporal  0.006899  prefrontal  0.006997   
2      visual  0.007120  prefrontal  0.006663    magnetic  0.006297   
3       using  0.006347       using  0.006310   resonance  0.006281   
4    magnetic  0.006195    magnetic  0.006292       using  0.006188   
5   resonance  0.006137   resonance  0.006181    response  0.005828   
6  prefrontal  0.005740     frontal  0.006083    anterior  0.005739   
7     frontal  0.005714    response  0.005602     frontal  0.005703   
8    response  0.005615    anterior  0.005403     control  0.005336   
9       gyrus  0.005260     network  0.005300   responses  0.005061   

      vision    Weight  
0       task  0.011901  
1     visual  0.007939  
2   parietal  0.006625  
3   temporal  0.006268  
4   magnetic  0.006172  
5      using  0.006139  
6  resonance  0.006079  
7    fr

## Decode reverse inference posterior probabilities (with uniform prior)

In [8]:
dfs = []
for i, name in enumerate(names):
    meta_file = 'temp/{0}_pFgA_given_pF=0.50.nii.gz'.format(name)
    results = decoder.decode_continuous(meta_file)
    results = results.sort_values(by=['Weight'], ascending=False)
    results[name] = results.index
    results = results[[name, 'Weight']].reset_index(drop=True)
    dfs.append(results)
df = pd.concat(dfs, axis=1)
print('Top terms:')
print df.head(10)

print('\nBottom terms:')
df = df.reindex(index=df.index[::-1])
print df.head(10)

Top terms:
        faces    Weight     emotion    Weight      reward    Weight  \
0        task  0.008976        task  0.008547        task  0.008908   
1    temporal  0.007790    temporal  0.007426  prefrontal  0.007110   
2       using  0.006676  prefrontal  0.006854       using  0.006583   
3  prefrontal  0.006381       using  0.006730    magnetic  0.006183   
4    magnetic  0.006053    magnetic  0.006245    temporal  0.006042   
5   resonance  0.005904   resonance  0.006039   resonance  0.006041   
6      visual  0.005749     frontal  0.005230     frontal  0.005212   
7      memory  0.005362      memory  0.005173     control  0.005178   
8     frontal  0.005098     network  0.005133      memory  0.005110   
9     network  0.005027       motor  0.005093       motor  0.005107   

       vision    Weight  
0        task  0.009657  
1    temporal  0.006834  
2       using  0.006596  
3      visual  0.006449  
4    magnetic  0.006172  
5   resonance  0.005997  
6  prefrontal  0.005766  

## Decode forward inference posterior probabilities (with uniform prior)

In [9]:
dfs = []
for i, name in enumerate(names):
    meta_file = 'temp/{0}_pAgF_given_pF=0.50.nii.gz'.format(name)
    results = decoder.decode_continuous(meta_file)
    results = results.sort_values(by=['Weight'], ascending=False)
    results[name] = results.index
    results = results[[name, 'Weight']].reset_index(drop=True)
    dfs.append(results)
df = pd.concat(dfs, axis=1)
print('Top terms:')
print df.head(10)

print('\nBottom terms:')
df = df.reindex(index=df.index[::-1])
print df.head(10)

Top terms:
        faces    Weight     emotion    Weight      reward    Weight  \
0        task  0.010830        task  0.010523        task  0.010581   
1    temporal  0.006720    temporal  0.006500  prefrontal  0.006479   
2      visual  0.006386  prefrontal  0.006287    magnetic  0.006229   
3       using  0.006264       using  0.006245       using  0.006194   
4    magnetic  0.006180    magnetic  0.006220   resonance  0.006167   
5   resonance  0.006097   resonance  0.006114     frontal  0.005864   
6  prefrontal  0.005896     frontal  0.006039    response  0.005674   
7     frontal  0.005880      visual  0.005586    temporal  0.005661   
8    response  0.005572    response  0.005565     control  0.005367   
9    parietal  0.005378    parietal  0.005388    parietal  0.005273   

       vision    Weight  
0        task  0.011361  
1      visual  0.006569  
2    temporal  0.006234  
3       using  0.006178  
4    parietal  0.006175  
5    magnetic  0.006169  
6   resonance  0.006070  

In [10]:
# Perform cleanup
rmtree('temp/')